In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

CATALOG_URL = "http://server:8080/catalog"
MANAGEMENT_URL = "http://server:8080/management"
DEMO_WAREHOUSE = "demo"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.5.2"
config = {
    "spark.sql.catalog.demo-catalog": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.demo-catalog.type": "rest",
    "spark.sql.catalog.demo-catalog.uri": CATALOG_URL,
    "spark.sql.catalog.demo-catalog.warehouse": DEMO_WAREHOUSE,
    "spark.sql.catalog.demo-catalog.io-impl": "org.apache.iceberg.aws.s3.S3FileIO",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.defaultCatalog": "demo-catalog",
    "spark.jars.packages": f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}"
}
spark_config = SparkConf().setMaster('local').setAppName("Iceberg-REST")
for k, v in config.items():
    spark_config = spark_config.set(k, v)

spark = SparkSession.builder.config(conf=spark_config).getOrCreate()

## Read and Write Tables

In [5]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS spark_demo")

DataFrame[]

In [6]:
spark.sql("SHOW NAMESPACES").toPandas()

,namespace
0,spark_demo


In [3]:
data = pd.DataFrame([[1, 'a-string', 2.2]], columns=['id', 'strings', 'floats'])
sdf = spark.createDataFrame(data)

In [7]:
sdf.writeTo("spark_demo.my_table").createOrReplace()

In [8]:
spark.sql("SELECT * FROM spark_demo.my_table").toPandas()

,id,strings,floats
0,1,a-string,2.2


In [12]:
spark.sql("create view spark_demo.myview as select id, strings from spark_demo.my_table")

DataFrame[]

In [15]:
spark.sql("select * from  spark_demo.myview3").toPandas()

,id,strings
0,1,a-string


In [17]:
spark.sql("DROP view spark_demo.myview")

DataFrame[]

In [19]:
spark.sql("DROP table spark_demo.my_table")

DataFrame[]